In [1]:
from os import system
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import Attention
import numpy as np
import sys
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from pickle import dump
from google.colab import drive
drive.mount('/content/drive')

np.set_printoptions(threshold=sys.maxsize)
# from scipy.signal import argrelextrema
# from statsmodels.nonparametric.kernel_regression import KernelReg
def find_MaxtimeStep(df):
    x = df.loc[df['TREND'].notnull()].index.tolist()
    max_distance =0
    for i in range (0,len(x)-1):
        if x[i+1]-x[i] >max_distance:
            max_distance = x[i+1]-x[i]
    return max_distance
def get_fixedSizeCandle(candles,md):
    candles_array = np.array(candles)
    Fixed_MaxSizeCandles = np.zeros((md,candles_array.shape[1]),dtype=float)
    Fixed_MaxSizeCandles[md-candles_array.shape[0]:,:candles_array.shape[1]] =candles_array
    return Fixed_MaxSizeCandles

def PreparetestData(path,Max_TimeStep):
    print ('Reading data...')
    excel_data = pd.read_excel(path)
    df = pd.DataFrame(excel_data, columns=['DATE', 'TIME', 'OPEN','HIGH','LOW','CLOSE','TREND'])
    print ('Reading data Complete!')
    print ('Creating Input Array ...')

    x=list()
    y=list()
    i=0
    candles = list()
    while i<len(df):
        candle =list()
        candle = np.zeros(4)
        if not(pd.isnull(df.at[i,'TREND'])):
            y.append(df.at[i,'TREND'])
            if len(candles)>0:
                    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
            candles = list()
        else:
          for j in range (0,4):
              candle[j] = df.iloc[i,j+2] -df.iloc[i-1,j+2]
            #candle[j] = df.iloc[i,j+2]
        candles.append(candle)
        i = i+1
    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
    x = np.array(x)
    y = np.array(y)
    # scaler = StandardScaler()
    # for i in range(0,x.shape[0]):
    #     x[i]= scaler.fit_transform(x[i])
    #x = scaler.fit_transform(x.reshape(-1, x.shape[-1])).reshape(x.shape)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    print ('Creating Input Array Complete!')
    return np.array(x),np.array(onehot_encoded)

def PrepareData(path,allORnum=0):
    print ('Reading data...')
    excel_data = pd.read_excel(path)
    df = pd.DataFrame(excel_data, columns=['DATE', 'TIME', 'OPEN','HIGH','LOW','CLOSE','TREND'])
    print ('Reading data Complete!')
    print ('Creating Input Array ...')

    Max_TimeStep = find_MaxtimeStep(df)
    x=list()
    y=list()
    i=0
    candles = list()
    while i<len(df):
        if len(x)> allORnum and allORnum!=0 :
          break
        candle =list()
        candle = np.zeros(4)
        if not(pd.isnull(df.at[i,'TREND'])):
            y.append(df.at[i,'TREND'])
            if len(candles)>0:
                    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
            candles = list()
        else:
          for j in range (0,4):
              candle[j] = df.iloc[i,j+2] -df.iloc[i-1,j+2]
            #candle[j] = df.iloc[i,j+2]
        candles.append(candle)
        i = i+1

    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
    x = np.array(x)
    y = np.array(y)
    scaler = StandardScaler()
    # for i in range(0,x.shape[0]):
    #   x[i]= scaler.fit_transform(x[i])
    #x = scaler.fit_transform(x.reshape(-1, x.shape[-1])).reshape(x.shape)
    #dump(scaler, open('/content/drive/MyDrive/Dataset/std_scaler.bin', 'wb'))
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    print ('Creating Input Array Complete!')
    return np.array(x),np.array(onehot_encoded),Max_TimeStep

def Create_Model(num_timesteps,num_features):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(num_timesteps, num_features)))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(16))
    model.add(Dense(6,activation= 'softmax'))
    return model

def Show_history_plot(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
def Create_Train_Test(x,y,test_Persent):

    X_Train = x[:int((100-test_Persent)*x.shape[0]/100),:,:]
    X_Test = x[(X_Train.shape[0]):,:,:]
    Y_Train = y[:X_Train.shape[0],:]
    Y_Test = y[X_Train.shape[0]:,:]
    return X_Train,X_Test,Y_Train,Y_Test

#set parameters
test_Persent = 15
Max_TimeStep=0
Max_TimeStep1=0
print('Preparing Data ...')
x,y,Max_TimeStep = PrepareData('/content/drive/MyDrive/Dataset/Final_DataSet_2400.xlsx')
#x1,y1, = PreparetestData('/content/drive/MyDrive/Test_Sample.xlsx',Max_TimeStep)
X_Train,X_Test,Y_Train,Y_Test = Create_Train_Test(x,y,test_Persent)
# scalers = {}
# for i in range(X_Train.shape[2]):
#     scalers[i] = StandardScaler()
#     X_Train[:, :, i] = scalers[i].fit_transform(X_Train[:, :, i]) 

# for i in range(X_Test.shape[2]):
#     X_Test[:, :, i] = scalers[i].transform(X_Test[:, :,i] )
scaler = StandardScaler()
for i in range(0,X_Train.shape[0]):
   X_Train[i]= scaler.fit_transform(X_Train[i])
scaler = StandardScaler()
for i in range(0,X_Test.shape[0]):
   X_Test[i]= scaler.fit_transform(X_Test[i])

print('Data is Ready ... (sample size:')
print(X_Train.shape)

model = Create_Model(Max_TimeStep,X_Train.shape[2])
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
history= model.fit(X_Train, Y_Train, epochs=25,  verbose=2)
model.save('/content/drive/MyDrive/Dataset/8_17_2022_LastModel')
#Show_history_plot(history)
#testPredict = model.predict(x1)
scores =model.evaluate(X_Test,Y_Test,verbose=0)
print(scores)
# classes = ['DownTrend','SideTrend','UpTrend']
# result=list()
# print(x1.shape)
# for i in range(0,len(testPredict)):
#   result.append(classes[np.argmax(testPredict[i])])
#   print(classes[np.argmax(testPredict[i])])
# excel_data = pd.read_excel('/content/drive/MyDrive/Test_Sample.xlsx')
# df = pd.DataFrame(excel_data, columns=['DATE', 'OPEN','HIGH','LOW','CLOSE','TREND'])
# df.rename(columns={'DATE': 'Date', 'OPEN': 'Open', 'HIGH': 'High', 'LOW': 'Low', 'CLOSE': 'Close'},inplace=True)
# Trend_Index = df.loc[df['TREND'].notnull()].index.tolist()
# for i in range(0,len(Trend_Index)-1):
#     Trend_df = pd.DataFrame(df.iloc[Trend_Index[i]:Trend_Index[i+1],:])
#     Trend_df.index = pd.DatetimeIndex(Trend_df['Date'])
#     fplt.plot(
#                 Trend_df,
#                 type='candle',
#                 title=result[i],
#                 ylabel='Price ($)'
#             )
#     fplt.show()
#print(model.evaluate(X_Test,Y_Test,verbose=0))


# classes = ['D','S','U']
# result = list()
# for i in range(0,len(testPredict)):
#   result.append([classes[np.argmax(Y_Test[i])],classes[np.argmax(testPredict[i])]])
# print(result)


Mounted at /content/drive
Preparing Data ...
Reading data...
Reading data Complete!
Creating Input Array ...
Creating Input Array Complete!
Data is Ready ... (sample size:
(2164, 700, 4)
Epoch 1/25
68/68 - 63s - loss: 1.1314 - accuracy: 0.6114 - 63s/epoch - 924ms/step
Epoch 2/25
68/68 - 58s - loss: 0.4788 - accuracy: 0.8480 - 58s/epoch - 854ms/step
Epoch 3/25
68/68 - 57s - loss: 0.3045 - accuracy: 0.9067 - 57s/epoch - 843ms/step
Epoch 4/25
68/68 - 57s - loss: 0.2263 - accuracy: 0.9288 - 57s/epoch - 844ms/step
Epoch 5/25
68/68 - 58s - loss: 0.2118 - accuracy: 0.9316 - 58s/epoch - 847ms/step
Epoch 6/25
68/68 - 57s - loss: 0.1956 - accuracy: 0.9358 - 57s/epoch - 837ms/step
Epoch 7/25
68/68 - 57s - loss: 0.2288 - accuracy: 0.9205 - 57s/epoch - 841ms/step
Epoch 8/25
68/68 - 56s - loss: 0.2314 - accuracy: 0.9191 - 56s/epoch - 826ms/step
Epoch 9/25
68/68 - 56s - loss: 0.1720 - accuracy: 0.9413 - 56s/epoch - 831ms/step
Epoch 10/25
68/68 - 58s - loss: 0.1623 - accuracy: 0.9473 - 58s/epoch - 851

[0.6003047227859497, 0.8041775226593018]
